# Comparing between different Sentimental Analaysis Models

Goal: To determine the best model on hand

Description:  I have a csv of headliners, and fed it to GrokAI to generate a list of sentimental Scores. This will be used as the benchmark in the comparison.

Steps outlined:
1. Setup the file "with_sentiment_100.csv" for comparison and briefly screen through the list for outliers
2. Run through the different models and run the data through them (We are interested to know if its positive or negative)
3. Compare with true values

Models:
- vader
- textblob
- flair
- roberta
- distilbert
- bertweet
- finbert
- deberta
- qwen llm

## 1. Setup testing file

In [ ]:
import pandas as pd
df = pd.read_csv("data/headlines_with_sentiment.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
headline = list(df['Headline'])
len(headline)

## 2. Run the list through different models

In [ ]:
# set up

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    if not isinstance(text, str) or text is None:
        return ""
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    test_sentence = " ".join(cleaned_tokens)
    return test_sentence

processed_headline = list(map(preprocess, headline))

In [ ]:
# 1. Prebuilt Vader sentiment package (NaiveBayes model) - Done

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
result_vader = []
threshold_upper = .05
threshold_lower = -.05
analyzer = SentimentIntensityAnalyzer()
for sentence in processed_headline:
    score = analyzer.polarity_scores(sentence)
    result_vader.append(score['compound'])
result_vader = ['POS' if s >= threshold_upper else 'NEG' if s <= threshold_lower else 'NEU' for s in result_vader]
print(result_vader[:10])
print(len(result_vader))
pd.DataFrame(result_vader).to_csv("data/result_vader.csv")

# to determine the threshold

In [ ]:
# 2. Prebuilt Textblob sentiment package - Done

from textblob import TextBlob
result_tb = []
threshold_upper = .05
threshold_lower = -.05
for sentence in processed_headline:
    result_tb.append(TextBlob(sentence).sentiment.polarity)
result_tb = ['POS' if s >= threshold_upper else 'NEG' if s <= threshold_lower else 'NEU' for s in result_tb]
print(result_tb[:10])
print(len(result_tb))
pd.DataFrame(result_tb).to_csv("data/result_tb.csv")

# to determine the threshold

In [ ]:
# 3. Prebuilt Flair sentiment package/Model - Done

from flair.data import Sentence
from flair.nn import Classifier
result_flair = []
tagger = Classifier.load('sentiment')

for sentence in processed_headline:
    sentence = Sentence(sentence)
    tagger.predict(sentence)
    value = sentence.labels[0].value
    result_flair.append(value)
result_flair = ['POS' if s == 'POSITIVE' else 'NEG' if s == 'NEGATIVE' else 'NEU' for s in result_flair]
print(result_flair[:10])
print(len(result_flair))
pd.DataFrame(result_flair).to_csv("data/result_flair.csv")

In [ ]:
# setup for HuggingFace Transformers

from transformers import pipeline, set_seed
set_seed(999)

In [ ]:
# RoBERTa - Done

classifier = pipeline('sentiment-analysis', model='cardiffnlp/twitter-roberta-base-sentiment-latest')

result_roberta = []
for sentence in processed_headline:
    temp = classifier(sentence)
    result_roberta.append(temp[0]['label'])
result_roberta = ['POS' if s == 'positive' else 'NEG' if s == 'negative' else 'NEU' for s in result_roberta]
print(result_roberta[:10])
print(len(result_roberta))
pd.DataFrame(result_roberta).to_csv("data/result_roberta.csv")

In [ ]:
# distilBERT - Done

classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

result_dis = []
for sentence in processed_headline:
    temp = classifier(sentence)
    result_dis.append(temp[0]['label'])
result_dis = ['POS' if s == 'POSITIVE' else 'NEG' if s == 'NEGATIVE' else 'NEU' for s in result_dis]
print(result_dis[:10])
print(len(result_dis))
pd.DataFrame(result_dis).to_csv("data/result_distilbert.csv")

In [ ]:
# Bertweet - Done

classifier = pipeline("text-classification", model="finiteautomata/bertweet-base-sentiment-analysis")

result_bertweet = []
for sentence in processed_headline:
    result_bertweet.append(classifier(sentence)[0]['label'])
print(result_bertweet[:10])
print(len(result_bertweet))
pd.DataFrame(result_bertweet).to_csv("data/result_bertweet.csv")

In [ ]:
# finBERT - Done

classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")

result_finbert = []
for sentence in processed_headline:
    temp = classifier(sentence)
    result_finbert.append(temp[0]['label'])
result_finbert = ['POS' if s == 'positive' else 'NEG' if s == 'negative' else 'NEU' for s in result_finbert]
print(result_finbert[:10])
print(len(result_finbert))
pd.DataFrame(result_finbert).to_csv("data/result_finbert.csv")

In [ ]:
# # LLM QWEN 8gb - Abandoned (ratelimit exceeded)
# import time
# from openai import OpenAI
#
# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key="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",
# )
#
# result_ai = []
# DELAY = 1  # Start with 1 second delay, adjust as needed
# for i, sentence in enumerate(processed_headline):
#     try:
#         completion = client.chat.completions.create(
#             extra_body={},
#             model="deepseek/deepseek-r1-0528-qwen3-8b:free",
#             messages=[{
#                 "role": "user",
#                 "content": f"Only give me a sentimental analysis value of NEG (negative), POS (positive), NEU (neutral) for the following sentence {sentence}. Dont add anything else to the output"
#             }]
#         )
#         result_ai.append(completion.choices[0].message.content)
#
#         # Print progress every 10 requests
#         if i % 10 == 0:
#             print(f"Processed {i+1}/{len(processed_headline)} requests")
#
#         # Add delay between requests
#         time.sleep(DELAY)
#
#     except Exception as e:
#         print(f"Error on request {i+1}: {str(e)}")
#         # If rate limited, increase delay and retry
#         DELAY += 1
#         time.sleep(DELAY)
#         continue
#
# print(result_ai[:10])
# print(len(result_ai))
# pd.DataFrame(result_ai).to_csv("data/result_ai.csv")

## Comparing with the benchmark

In [ ]:
# visualising the data

